# Low Level APIs
Follow [Programmer's Guide](https://www.tensorflow.org/programmers_guide/) to learn TensorFlow basics

## Introduction

### TensorFlow Core Walkthrough

In [ ]:
# upgrade stuffs
# tensorflow upgrade could be slow if running in notebook. try in BASH and see if it's faster.

# upgrade tensorflow before running this notebook
# pip install -q --upgrade tensorflow
# in emulator the above command can be prefixed with !
# this should upgrade tensorflow to 1.7.0

!pip install -q --upgrade tensorflow

In [26]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib
from matplotlib import pyplot as plt

import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe

print("matplotlib version: {}", matplotlib.__version__)
print("numpy version: {}", np.__version__)
print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

matplotlib version: {} 2.2.2
numpy version: {} 1.14.2
TensorFlow version: 1.7.0
Eager execution: False


In [ ]:
# turn on eager execution in this box
# do this only when necessary
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

### Feeding


In [ ]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)
sess = tf.Session()
# the next line could cause errer if eager execution is enabled.
if not tf.executing_eagerly(): print(sess.run([node1, node2]))
# node3 = tf.add(node1, node2)
node3 = node1 + node2
print("node3:", node3)
if not tf.executing_eagerly(): print("sess.run(node3):", sess.run(node3))

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

In [ ]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

In [ ]:
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))

In [ ]:
!head -n5 {train_dataset_fp}

In [ ]:
!ls -al

In [ ]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

# event file name is like events.out.tfevents.{timestamp}.{hostname}

In [ ]:
# must run this in BASH
# otherwise cannot cancel

!tensorboard --logdir .

### Datasets


In [ ]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]

slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

In [ ]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

### Layers


In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

In [ ]:
# Use the shortcut function tf.layers.dense()
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

### Feature columns


In [ ]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [ ]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

print(sess.run(inputs))

In [ ]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

print(sess.run(y_pred))

print(sess.run(loss))

for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

print(sess.run(y_pred))